In [ ]:
# Install biopython
! pip install biopython

In [ ]:
from Bio import Entrez, SeqIO

In [ ]:
Entrez.email = "fpichardom@yahoo.com"

In [ ]:
# Search param
ORGN =  'Pereskia'
RETMAX = 300

In [ ]:
search_handle = Entrez.esearch(
    db='nucleotide', # NCBI database
    term = f'{ORGN}[ORGN]', # Search term as a string
    idtype = "acc", 
    retmax = RETMAX
)                               

In [ ]:
#search_handle.read()

In [ ]:
search_results = Entrez.read(search_handle)

In [ ]:
search_results

In [ ]:
id_list_str = ",".join(search_results['IdList'])

In [ ]:
fetch_handle = Entrez.efetch(
    db='nucleotide',
    id=id_list_str,
    retmode='xml'
)

In [ ]:
fetch_results = Entrez.read(fetch_handle)

In [ ]:
len(fetch_results)

In [ ]:
type(fetch_results)

In [ ]:
fetch_results[0]

In [ ]:
entry = fetch_results[0]

In [ ]:
for key, value in entry.items():
    print(key, type(value))
    print("+==================================+")

In [ ]:
gb_record = {
    "accession_id": entry['GBSeq_accession-version'],
    "taxonomy": {
        "taxon_name": entry['GBSeq_organism']
    },
    "features": [],
    "sequence": entry['GBSeq_sequence']
}

In [ ]:
features = entry['GBSeq_feature-table']

In [ ]:
for feature in features:
    if feature['GBFeature_key'] != "source":
        feat_entry = {
            "feature_key": feature['GBFeature_key'],
            "location": feature['GBFeature_location'],
            "qualifier_name": feature['GBFeature_quals'][0]['GBQualifier_name'],
            "qualifier_value": feature['GBFeature_quals'][0]['GBQualifier_value']
    }
        gb_record['features'].append(feat_entry)

In [ ]:
gb_record

# Combine All

In [ ]:
from Bio import Entrez, SeqIO

In [ ]:
Entrez.email = "fpichardom@yahoo.com"

In [ ]:
# Search param
ORGN =  'Pereskia'
RETMAX = 300

In [ ]:
search_handle = Entrez.esearch(
    db='nucleotide', # NCBI database
    term = f'{ORGN}[ORGN]', # Search term as a string
    idtype = "acc", 
    retmax = RETMAX
)                               

In [ ]:
search_results = Entrez.read(search_handle)

In [ ]:
id_list_str = ",".join(search_results['IdList'])

In [ ]:
fetch_handle = Entrez.efetch(
    db='nucleotide',
    id=id_list_str,
    retmode='xml'
)

In [ ]:
fetch_results = Entrez.read(fetch_handle)

In [ ]:
len(fetch_results)

In [ ]:

all_records = []
for entry in fetch_results:
    
    try:
        gb_record = {
            "accession_id": entry['GBSeq_accession-version'],
            "taxonomy": {
                "taxon_name": entry['GBSeq_organism']
            },
            "features": [],
            "sequence": entry['GBSeq_sequence']
        }

        features = entry['GBSeq_feature-table']

        for feature in features:
            if feature['GBFeature_key'] != "source":
                feat_entry = {
                    "feature_key": feature['GBFeature_key'],
                    "location": feature['GBFeature_location'],
                    "qualifier_name": feature['GBFeature_quals'][0]['GBQualifier_name'],
                    "qualifier_value": feature['GBFeature_quals'][0]['GBQualifier_value']
                }
                gb_record['features'].append(feat_entry)
        all_records.append(gb_record)
    
    except KeyError:
        print(entry['GBSeq_accession-version'])


In [ ]:
len(all_records)

In [ ]:
all_records[0:8]

In [ ]:
# How many empty features
count = 0
for record in all_records:
    if record['features']:
        count+= 1
    

In [ ]:
count

In [ ]:
unnatural = []

In [ ]:
for record in all_records:
    if record['features']:
        new = {
            "taxon_name": record['taxonomy']['taxon_name'],
            "marker": ""
        }
        for feature in record['features']:
            new['marker'] = feature['qualifier_value']
            unnatural.append(new)

In [ ]:
len(unnatural)

In [ ]:
unnatural[0:2]

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame.from_records(unnatural)

In [ ]:
df.head()

In [ ]:
accum_markers = df.groupby('taxon_name').agg(lambda col: list(set(col))).reset_index()

In [ ]:
accum_markers['count'] = accum_markers.apply(lambda x: len(x['marker']), axis=1)

In [ ]:
accum_markers

In [ ]:
import json

In [ ]:
# Write basic json
with open('gb_results.json', 'w') as out:
    json.dump(all_records, out)

In [ ]:
# Write prettier json
with open('gb_results.json', 'w') as out:
    json.dump(all_records, out, indent= 4, sort_keys=True)